In [2]:
import pandas as pd
import numpy as np

# --- 1. Load the Datasets ---

# Users Table: Represents the "Who"
# Contains demographic info, signup dates, and job titles.
df_users = pd.read_csv('users.csv')

# Events Table: Represents the "What" (Behavior)
# Critical for PLG. Tracks clicks, feature usage, and session times.
df_events = pd.read_csv('events.csv')

# Subscriptions Table: Represents the "Value" (Revenue)
# Contains payment history, plan types (Free/Pro), and churn data.
df_subs = pd.read_csv('subscriptions.csv')

# Marketing Spend Table: Represents the "Cost" (Acquisition)
# Tracks how much was spent on ads/channels to acquire these users.
df_marketing = pd.read_csv('marketing_spend.csv')

# --- 2. Inspect the Data ---

# Helper function to print info cleanly
def inspect_dataframe(df, name):
    print(f"--- INSPECTING: {name} ---")
    print(f"Shape (Rows, Columns): {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\nFirst 5 Rows:")
    display(df.head()) # 'display()' works best in Jupyter
    print("\n" + "="*50 + "\n")

# Run inspection for all files
inspect_dataframe(df_users, "Users Data")
inspect_dataframe(df_events, "Events Data")
inspect_dataframe(df_subs, "Subscriptions Data")
inspect_dataframe(df_marketing, "Marketing Spend Data")

--- INSPECTING: Users Data ---
Shape (Rows, Columns): (100000, 5)
Columns: ['user_id', 'signup_date', 'country', 'persona', 'acquisition_channel']

First 5 Rows:


,user_id,signup_date,country,persona,acquisition_channel
0,U00001,2024-02-08,India,team,email
1,U00002,2024-02-21,UK,student,organic
2,U00003,2024-01-29,India,student,paid_ads
3,U00004,2024-01-15,USA,team,paid_ads
4,U00005,2024-02-12,India,student,email




--- INSPECTING: Events Data ---
Shape (Rows, Columns): (277009, 5)
Columns: ['user_id', 'event_time', 'event_name', 'platform', 'session_id']

First 5 Rows:


,user_id,event_time,event_name,platform,session_id
0,U00001,2024-02-08 00:00:00,signup,web,S_U00001_1
1,U00001,2024-02-08 01:00:00,code_run,web,S_U00001_1
2,U00001,2024-02-09 00:00:00,deploy,web,S_U00001_2
3,U00001,2024-02-18 00:00:00,upgrade,web,S_U00001_4
4,U00002,2024-02-21 00:00:00,signup,web,S_U00002_1




--- INSPECTING: Subscriptions Data ---
Shape (Rows, Columns): (18021, 5)
Columns: ['user_id', 'plan_type', 'start_date', 'mrr', 'churned']

First 5 Rows:


,user_id,plan_type,start_date,mrr,churned
0,U00002,pro,2024-01-23,40,False
1,U00003,pro,2024-01-24,20,True
2,U00012,team,2024-01-13,40,False
3,U00026,pro,2024-01-28,20,False
4,U00027,team,2024-01-17,20,False




--- INSPECTING: Marketing Spend Data ---
Shape (Rows, Columns): (244, 3)
Columns: ['date', 'channel', 'spend']

First 5 Rows:


,date,channel,spend
0,2024-01-01,organic,242
1,2024-01-01,paid_ads,278
2,2024-01-01,email,307
3,2024-01-01,referral,289
4,2024-01-02,organic,374


In [4]:
# --- 1. Data Cleaning & Type Conversion ---

# Convert timestamp columns to datetime objects to allow date math
df_events['event_time'] = pd.to_datetime(df_events['event_time'])
df_users['signup_date'] = pd.to_datetime(df_users['signup_date'])

# --- 2. Create User-Level Aggregations ---

# Calculate counts for specific high-value actions
# We use crosstab to pivot the event names into columns (e.g., 'code_run', 'deploy')
# We assume the column containing event types is named 'event_name'
event_type_counts = pd.crosstab(df_events['user_id'], df_events['event_name'])

# Select only the specific events we care about and rename them
target_events = ['code_run', 'deploy', 'collaborate']

# Ensure columns exist (handling case where an event might not be in the sample)
for event in target_events:
    if event not in event_type_counts.columns:
        event_type_counts[event] = 0

# Rename columns for clarity (e.g., 'code_run' -> 'code_run_count')
event_type_counts = event_type_counts[target_events].add_suffix('_count')

# Calculate general activity metrics (Total events, Active days, Last active date)
# lambda x: x.dt.date.nunique() counts the number of unique days a user was active
activity_stats = df_events.groupby('user_id').agg(
    total_events=('event_time', 'count'),
    active_days=('event_time', lambda x: x.dt.date.nunique()),
    last_event_date=('event_time', 'max')
)

# --- 3. Merge Features with Users DataFrame ---

# Start with df_users as the base (Left Join)
# This ensures we keep users who signed up but have zero events
df_features = df_users.merge(activity_stats, on='user_id', how='left')
df_features = df_features.merge(event_type_counts, on='user_id', how='left')

# --- 4. Final Feature Calculations & Cleanup ---

# Fill NaN values with 0 (Users with no events have 0 counts/days)
fill_cols = ['total_events', 'active_days', 'code_run_count', 'deploy_count', 'collaborate_count']
df_features[fill_cols] = df_features[fill_cols].fillna(0)

# Calculate 'days_since_signup' (Time from Signup to their Last Event)
# This represents the user's "lifetime" on the platform so far
df_features['days_since_signup'] = (df_features['last_event_date'] - df_features['signup_date']).dt.days

# Handle users with no events (days_since_signup will be NaT/NaN)
df_features['days_since_signup'] = df_features['days_since_signup'].fillna(0)

# --- 5. Inspect the Final Dataset ---

print(f"--- Final Feature DataFrame ---")
print(f"Shape: {df_features.shape}")
print(f"Columns: {list(df_features.columns)}")
print("\nFirst 5 rows:")
display(df_features.head())

--- Final Feature DataFrame ---
Shape: (100000, 12)
Columns: ['user_id', 'signup_date', 'country', 'persona', 'acquisition_channel', 'total_events', 'active_days', 'last_event_date', 'code_run_count', 'deploy_count', 'collaborate_count', 'days_since_signup']

First 5 rows:


,user_id,signup_date,country,persona,acquisition_channel,total_events,active_days,last_event_date,code_run_count,deploy_count,collaborate_count,days_since_signup
0,U00001,2024-02-08,India,team,email,4,3,2024-02-18 00:00:00,1,1,0,10
1,U00002,2024-02-21,UK,student,organic,3,2,2024-02-22 00:00:00,1,1,0,1
2,U00003,2024-01-29,India,student,paid_ads,3,2,2024-01-30 00:00:00,1,1,0,1
3,U00004,2024-01-15,USA,team,paid_ads,4,3,2024-01-17 00:00:00,1,1,1,2
4,U00005,2024-02-12,India,student,email,2,1,2024-02-12 01:00:00,1,0,0,0


In [5]:
# --- 1. Identify Converted Users from Multiple Sources ---

# Source A: Users who exist in the subscriptions table (The "Revenue" Truth)
# These users have successfully processed a payment or started a plan.
subscribed_user_ids = set(df_subs['user_id'])

# Source B: Users who triggered an "upgrade" event (The "Behavioral" Truth)
# We look for explicit intent actions in the event logs.
# Note: We assume the event name is 'upgrade' based on standard tracking.
upgrade_event_users = set(df_events[df_events['event_name'] == 'upgrade']['user_id'])

# Combine both sets (Union)
# This captures users who Paid OR tried to Upgrade.
high_intent_user_ids = subscribed_user_ids.union(upgrade_event_users)

# --- 2. Create the Binary Target Label ---

# Create a new column 'high_intent'
# .isin() checks if the user is in our high_intent set.
# .astype(int) converts the boolean (True/False) to binary (1/0).
df_features['high_intent'] = df_features['user_id'].isin(high_intent_user_ids).astype(int)

# --- 3. Inspect the Label Distribution ---

print("--- Target Distribution (High Intent vs. Low Intent) ---")
# normalize=True gives us the percentage, which is crucial for checking class imbalance.
distribution = df_features['high_intent'].value_counts(normalize=True) * 100
counts = df_features['high_intent'].value_counts()

print(f"Total Users: {len(df_features)}")
print(f"High Intent (1): {counts.get(1, 0)} users ({distribution.get(1, 0):.2f}%)")
print(f"Low Intent  (0): {counts.get(0, 0)} users ({distribution.get(0, 0):.2f}%)")

print("\n" + "="*50 + "\n")

# --- 4. Show Example Rows ---

print("--- Example Rows: Features + Target ---")
# We sample 5 rows to ensure we see the new column
display(df_features[['user_id', 'total_events', 'deploy_count', 'days_since_signup', 'high_intent']].head())

--- Target Distribution (High Intent vs. Low Intent) ---
Total Users: 100000
High Intent (1): 33377 users (33.38%)
Low Intent  (0): 66623 users (66.62%)


--- Example Rows: Features + Target ---


,user_id,total_events,deploy_count,days_since_signup,high_intent
0,U00001,4,1,10,1
1,U00002,3,1,1,1
2,U00003,3,1,1,1
3,U00004,4,1,2,0
4,U00005,2,0,0,0


In [6]:
# --- 1. Define the Business Logic (Heuristic Score) ---
# We start with a base score of 0 for everyone
df_features['intent_score'] = 0

# Rule 1: Activation (Deploys are high value) -> +2 Points
df_features.loc[df_features['deploy_count'] >= 1, 'intent_score'] += 2

# Rule 2: Engagement (Running code shows effort) -> +1 Point
df_features.loc[df_features['code_run_count'] >= 2, 'intent_score'] += 1

# Rule 3: Virality (Collaboration is sticky) -> +1 Point
df_features.loc[df_features['collaborate_count'] >= 1, 'intent_score'] += 1

# Rule 4: Habit (Coming back multiple days) -> +1 Point
df_features.loc[df_features['active_days'] >= 3, 'intent_score'] += 1

# Max possible score is 5. No need to normalize for this simple baseline.

# --- 2. Evaluate the Score's Effectiveness ---

print("--- Average Score by Outcome ---")
# Compare the average score of users who upgraded vs. those who didn't
avg_scores = df_features.groupby('high_intent')['intent_score'].mean()
print(f"Low Intent Users (0): {avg_scores[0]:.2f}")
print(f"High Intent Users (1): {avg_scores[1]:.2f}")
print("-" * 30)

# --- 3. Create the Summary Table ---

# Group by the score to see how conversion rate changes as score increases
score_summary = df_features.groupby('intent_score').agg(
    user_count=('user_id', 'count'),
    converted_users=('high_intent', 'sum'),
    conversion_rate=('high_intent', 'mean')
).reset_index()

# Format the conversion rate as a percentage for readability
score_summary['conversion_rate_pct'] = (score_summary['conversion_rate'] * 100).round(1).astype(str) + '%'

print("\n--- Conversion Performance by Score Bucket ---")
display(score_summary[['intent_score', 'user_count', 'conversion_rate_pct']])

--- Average Score by Outcome ---
Low Intent Users (0): 1.28
High Intent Users (1): 1.84
------------------------------

--- Conversion Performance by Score Bucket ---


,intent_score,user_count,conversion_rate_pct
0,0,41152,28.7%
1,1,6878,18.4%
2,2,30489,22.9%
3,3,7095,100.0%
4,4,14386,43.4%


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# --- 1. Select Features and Target ---

# We only use numeric features for this baseline to keep it simple
feature_cols = [
    'total_events',
    'active_days',
    'code_run_count',
    'deploy_count',
    'collaborate_count',
    'days_since_signup'
]

X = df_features[feature_cols]
y = df_features['high_intent']

# --- 2. Split Data (Train/Test) ---

# Stratify=y ensures both train and test sets have the same % of converted users
# random_state=42 ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training on {X_train.shape[0]} users, Testing on {X_test.shape[0]} users.")

# --- 3. Train the Model ---

# We increase max_iter slightly to ensure the solver converges
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# --- 4. Evaluate Performance ---

# Predict class (0 or 1)
y_pred = model.predict(X_test)
# Predict probability (0.0 to 1.0) - needed for AUC
y_prob = model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print(f"\n--- Model Performance ---")
print(f"Accuracy: {accuracy:.4f} (How often is the model correct?)")
print(f"ROC-AUC:  {auc:.4f} (How well does it separate High vs Low intent?)")

# --- 5. Interpret Coefficients ---

# Create a DataFrame to verify which features drive conversion
coef_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print("\n--- Feature Importance (Coefficients) ---")
display(coef_df)

Training on 70000 users, Testing on 30000 users.

--- Model Performance ---
Accuracy: 0.8519 (How often is the model correct?)
ROC-AUC:  0.7776 (How well does it separate High vs Low intent?)

--- Feature Importance (Coefficients) ---


,Feature,Coefficient
1,active_days,4.056072
0,total_events,2.025617
5,days_since_signup,0.278179
2,code_run_count,-2.030455
3,deploy_count,-6.318341
4,collaborate_count,-6.466516
